# Skip next few cells - These cells are trying to access the PFAM database (unsuccessfully)


> Add blockquote



In [ ]:
# Second cell - Import libraries and define functions
import requests
from Bio import AlignIO, Phylo
from Bio.Phylo.TreeConstruction import DistanceCalculator, DistanceTreeConstructor
import subprocess
import tempfile
from google.colab import files
import matplotlib.pyplot as plt
import io
import os
import urllib3
from io import StringIO
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
def fetch_pfam_sequences(pfam_id):
    """Download sequences from Pfam for a given family ID using REST API"""
    # Using the REST API endpoint
    url = f"https://rest.ensembl.org/family/id/{pfam_id}?content-type=application/json"

    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()

            # Extract sequences and create FASTA format
            sequences = []
            for member in data.get('members', []):
                seq_id = member.get('species') + '/' + member.get('id')
                sequence = member.get('sequence')
                if sequence:
                    sequences.append(f">{seq_id}\n{sequence}\n")

            if not sequences:
                raise ValueError("No sequences found in the response")

            # Save sequences to a FASTA file
            fasta_content = "".join(sequences)
            with tempfile.NamedTemporaryFile(mode='w', suffix='.fasta', delete=False) as temp_file:
                temp_file.write(fasta_content)
                return temp_file.name
        else:
            print(f"HTTP Status Code: {response.status_code}")
            raise ValueError(f"Failed to fetch sequences: Status code {response.status_code}")

    except Exception as e:
        print(f"Error during download: {str(e)}")
        raise

def realign_sequences(fasta_file):
    """Realign sequences using MAFFT via subprocess"""
    output_file = fasta_file.replace('.fasta', '_aligned.fasta')

    # Run MAFFT using subprocess
    cmd = ['mafft', '--auto', fasta_file]
    try:
        result = subprocess.run(cmd,
                              capture_output=True,
                              text=True,
                              check=True)

        # Write the aligned sequences to file
        with open(output_file, 'w') as handle:
            handle.write(result.stdout)

        return output_file

    except subprocess.CalledProcessError as e:
        print(f"MAFFT alignment failed: {e}")
        print(f"MAFFT error output: {e.stderr}")
        raise
    except Exception as e:
        print(f"An error occurred while running MAFFT: {e}")
        raise

def build_phylogenetic_tree(aligned_fasta):
    """Build phylogenetic tree using distance-based method"""
    alignment = AlignIO.read(aligned_fasta, 'fasta')

    # Calculate distance matrix
    calculator = DistanceCalculator('identity')
    dm = calculator.get_distance(alignment)

    # Build tree
    constructor = DistanceTreeConstructor()
    tree = constructor.build_tree(dm)

    return tree

def visualize_tree(tree):
    """Create and display a matplotlib figure of the tree"""
    plt.figure(figsize=(20, 20))  # Made figure larger
    Phylo.draw(tree, do_show=False,
               label_func=lambda x: x.name[:30],  # Trim long names
               leaf_font_size=8)  # Smaller font for leaf labels
    plt.title("Protein Family Phylogenetic Tree")
    plt.axis('off')
    plt.show()

def analyze_pfam_family(pfam_id):
    """Main workflow function"""
    try:
        # Download sequences
        print(f"Downloading sequences for {pfam_id}...")
        fasta_file = fetch_pfam_sequences(pfam_id)

        # Print some information about the downloaded sequences
        with open(fasta_file, 'r') as f:
            seq_count = sum(1 for line in f if line.startswith('>'))
        print(f"\nNumber of sequences downloaded: {seq_count}")

        # Realign using MAFFT
        print("\nRealigning sequences...")
        aligned_file = realign_sequences(fasta_file)

        # Build tree
        print("\nBuilding phylogenetic tree...")
        tree = build_phylogenetic_tree(aligned_file)

        # Save tree to Newick format
        tree_file = f"{pfam_id}_tree.newick"
        Phylo.write(tree, tree_file, 'newick')
        print(f"\nTree saved as {tree_file}")

        # Visualize tree
        print("\nGenerating tree visualization...")
        visualize_tree(tree)

        # Download files
        print("\nPreparing files for download...")
        files.download(tree_file)
        files.download(aligned_file)

        # Cleanup temporary files
        if os.path.exists(fasta_file):
            os.remove(fasta_file)

    except Exception as e:
        print(f"Error: {str(e)}")

In [ ]:
# Try with different protein families
protein_families = [
    "PF00042",  # Globin
    "PF00069",  # Protein kinase domain
    "PF00071"   # Ras family
]

for pfam_id in protein_families:
    print(f"\nAnalyzing {pfam_id}...")
    analyze_pfam_family(pfam_id)


Analyzing PF00042...
HTTP Status Code: 404
Error during download: Failed to fetch sequences: Status code 404
Error: Failed to fetch sequences: Status code 404

Analyzing PF00069...
HTTP Status Code: 404
Error during download: Failed to fetch sequences: Status code 404
Error: Failed to fetch sequences: Status code 404

Analyzing PF00071...
HTTP Status Code: 404
Error during download: Failed to fetch sequences: Status code 404
Error: Failed to fetch sequences: Status code 404


# Start here - Use UniProtein



In [1]:
# First cell - Installation commands
!apt-get update
!apt-get install -y mafft
!pip install biopython requests

zsh:1: command not found: apt-get
zsh:1: command not found: apt-get
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.2.3-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 33.1 MB/s eta 0:00:00
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached certifi-2024.8.30-py3-none-any.whl (167 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached urllib3-2.2.3-py3-none-any.whl (126 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 47.3 MB/s eta 0:00:00


In [4]:
import requests
from Bio import AlignIO, Phylo, SeqIO
from Bio.Phylo.TreeConstruction import DistanceCalculator, DistanceTreeConstructor
import subprocess
import tempfile
from google.colab import files
import matplotlib.pyplot as plt
import os
from io import StringIO
import urllib.parse

In [11]:
def fetch_uniprot_sequences(keyword, max_sequences=50):
    """Download sequences from UniProt for a given protein family"""
    # Encode the query properly for UniProt API
    encoded_keyword = urllib.parse.quote(keyword)

    # UniProt REST API URL
    url = f"https://rest.uniprot.org/uniprotkb/search?query={encoded_keyword}&format=fasta&size={max_sequences}"

    try:
        print(f"Fetching from URL: {url}")
        response = requests.get(url)

        if response.status_code == 200:
            # Check if we got any sequences
            if len(response.text.strip()) == 0:
                raise ValueError("No sequences found in the response")

            # Save sequences to a FASTA file
            with tempfile.NamedTemporaryFile(mode='w', suffix='.fasta', delete=False) as temp_file:
                temp_file.write(response.text)
                return temp_file.name
        else:
            print(f"HTTP Status Code: {response.status_code}")
            print(f"Response content: {response.text[:200]}")  # Print first 200 chars of response
            raise ValueError(f"Failed to fetch sequences: Status code {response.status_code}")

    except Exception as e:
        print(f"Error during download: {str(e)}")
        raise

def realign_sequences(fasta_file):
    """Realign sequences using MAFFT"""
    output_file = fasta_file.replace('.fasta', '_aligned.fasta')

    cmd = ['mafft', '--auto', fasta_file]
    try:
        result = subprocess.run(cmd,
                              capture_output=True,
                              text=True,
                              check=True)

        with open(output_file, 'w') as handle:
            handle.write(result.stdout)

        return output_file

    except subprocess.CalledProcessError as e:
        print(f"MAFFT alignment failed: {e}")
        print(f"MAFFT error output: {e.stderr}")
        raise
    except Exception as e:
        print(f"An error occurred while running MAFFT: {e}")
        raise

def build_phylogenetic_tree(aligned_fasta):
    """Build phylogenetic tree using distance-based method"""
    try:
        # Read the alignment
        alignment = AlignIO.read(aligned_fasta, 'fasta')
        print(f"Successfully read alignment with {len(alignment)} sequences")

        # Calculate distance matrix using 'identity' model
        calculator = DistanceCalculator('identity')
        print("Calculating distance matrix...")
        dm = calculator.get_distance(alignment)

        # Build tree using UPGMA method
        print("Building tree using UPGMA method...")
        constructor = DistanceTreeConstructor(calculator, 'upgma')
        tree = constructor.build_tree(alignment)

        return tree

    except Exception as e:
        print(f"Error in build_phylogenetic_tree: {str(e)}")
        print(f"Error type: {type(e)}")
        raise

def visualize_tree(tree, title):
    """Create and display a matplotlib figure of the tree"""
    plt.figure(figsize=(20, 20))
    # Fix the font size issue by setting rcParams
    plt.rcParams['font.size'] = 8

    Phylo.draw(tree,
               do_show=False,
               label_func=lambda x: x.name[:30])  # Trim long names

    plt.title(title, fontsize=14)
    plt.axis('off')
    plt.tight_layout()
    plt.show()

def analyze_protein_family(keyword, title):
    """Main workflow function with sequence saving"""
    try:
        # Download sequences
        print(f"Downloading sequences for {keyword}...")
        fasta_file = fetch_uniprot_sequences(keyword)
        # Save a copy of the original sequences
        original_sequences_file = f"{title.replace(' ', '_')}_sequences.fasta"
        with open(fasta_file, 'r') as source:
            with open(original_sequences_file, 'w') as dest:
                dest.write(source.read())

        # Print sequence information
        sequences = list(SeqIO.parse(fasta_file, "fasta"))
        print(f"\nNumber of sequences downloaded: {len(sequences)}")
        print("\nSequence IDs:")
        for seq in sequences:
            print(f"{seq.id}: {seq.description}")

        # Download the original sequences
        files.download(original_sequences_file)

        print(f"\nOriginal sequences saved to: {original_sequences_file}")

        if len(sequences) < 3:
            print("Not enough sequences found for analysis")
            return

        # Realign using MAFFT
        print("\nRealigning sequences...")
        aligned_file = realign_sequences(fasta_file)

        # Build tree
        print("\nBuilding phylogenetic tree...")
        tree = build_phylogenetic_tree(aligned_file)

        # Save tree to Newick format
        safe_filename = "".join(x for x in keyword if x.isalnum() or x in "._- ")
        tree_file = f"{safe_filename}_tree.newick"
        Phylo.write(tree, tree_file, 'newick')
        print(f"\nTree saved as {tree_file}")

        # Visualize tree
        print("\nGenerating tree visualization...")
        visualize_tree(tree, title)

        # Download files
        print("\nPreparing files for download...")
        files.download(tree_file)
        files.download(aligned_file)

        # Cleanup
        if os.path.exists(fasta_file):
            os.remove(fasta_file)

    except Exception as e:
        print(f"Error: {str(e)}")

In [12]:
# Analyze different protein families using correct UniProt search syntax
protein_families = [
    ("protein_name:globin AND reviewed:true", "Globin Family Tree"),
    ("protein_name:insulin receptor kinase AND reviewed:true", "Insulin Receptor Kinase Tree"),
    ("gene:RAS AND reviewed:true", "Ras Family Tree")
]

# Let's try just one family first to make sure it works
keyword, title = protein_families[0]  # Try with globin first
print(f"\nAnalyzing {title}...")
analyze_protein_family(keyword, title)


Analyzing Globin Family Tree...
Fetching from URL: https://rest.uniprot.org/uniprotkb/search?query=protein_name%3Aglobin%20AND%20reviewed%3Atrue&format=fasta&size=50
File saved at: /Users/magizhshankar/Documents/CS 466/final_project/output.fasta
None
Error: division by zero


In [5]:
from Bio import SeqIO

# To print sequences from a specific FASTA file
file_name = "Globin_Family_Tree_sequences.fasta"  # or whatever your file is named

# Print all sequences with their IDs and sequences
print("Printing sequences from", file_name)
for record in SeqIO.parse(file_name, "fasta"):
    print("\nSequence ID:", record.id)
    print("Description:", record.description)
    print("Sequence:", record.seq)
    print("Length:", len(record.seq))
    print("-" * 50)

# Or if you want just the first few sequences:
print("\nFirst 3 sequences:")
for i, record in enumerate(SeqIO.parse(file_name, "fasta")):
    if i < 3:  # Only print first 3
        print("\nSequence ID:", record.id)
        print("Sequence:", record.seq)
    else:
        break

Printing sequences from Globin_Family_Tree_sequences.fasta

Sequence ID: sp|P02100|HBE_HUMAN
Description: sp|P02100|HBE_HUMAN Hemoglobin subunit epsilon OS=Homo sapiens OX=9606 GN=HBE1 PE=1 SV=2
Sequence: MVHFTAEEKAAVTSLWSKMNVEEAGGEALGRLLVVYPWTQRFFDSFGNLSSPSAILGNPKVKAHGKKVLTSFGDAIKNMDNLKPAFAKLSELHCDKLHVDPENFKLLGNVMVIILATHFGKEFTPEVQAAWQKLVSAVAIALAHKYH
Length: 147
--------------------------------------------------

Sequence ID: sp|P02042|HBD_HUMAN
Description: sp|P02042|HBD_HUMAN Hemoglobin subunit delta OS=Homo sapiens OX=9606 GN=HBD PE=1 SV=2
Sequence: MVHLTPEEKTAVNALWGKVNVDAVGGEALGRLLVVYPWTQRFFESFGDLSSPDAVMGNPKVKAHGKKVLGAFSDGLAHLDNLKGTFSQLSELHCDKLHVDPENFRLLGNVLVCVLARNFGKEFTPQMQAAYQKVVAGVANALAHKYH
Length: 147
--------------------------------------------------

Sequence ID: sp|P02008|HBAZ_HUMAN
Description: sp|P02008|HBAZ_HUMAN Hemoglobin subunit zeta OS=Homo sapiens OX=9606 GN=HBZ PE=1 SV=2
Sequence: MSLTKTERTIIVSMWAKISTQADTIGTETLERLFLSHPQTKTYFPHFDLHPGSAQLRAHGSKVVAAVGDAVKSIDDIGGALSKLSE